In [1]:
from imutils.video import VideoStream
from imutils import face_utils
import datetime
import argparse
import imutils
import time
import dlib
import cv2
import glob
import sys
import os
import shutil 
import math
import numpy as np
from albumentations.augmentations import transforms
import skimage
from tqdm import tqdm

from skimage.transform import resize
from sklearn.model_selection import train_test_split
from scipy import stats as s
import scipy.misc
from PIL import Image


import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.layers import Dense, InputLayer, Dropout, Flatten, Activation, GRU, Input, Conv3D, MaxPooling3D, GlobalMaxPooling3D, ZeroPadding3D, AveragePooling3D
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Convolution2D, ZeroPadding2D, Bidirectional, TimeDistributed, GlobalAveragePooling2D
from keras.layers.recurrent import LSTM
from keras.utils import np_utils
from keras.preprocessing import image, sequence
from keras.callbacks import ModelCheckpoint


#from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import optimizers

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
img_black = np.zeros((25,50,3))

base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

classes = 'Begin, Choose, Connection, Navigation, Next, Previous, Start, Stop, Hello, Web'
classes = classes.split(', ')

predictor_path = 'assests/predictors/shape_predictor_68_face_landmarks.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

train_people = 'F01, F02, F04, F05, F06, F07, F08, F09, M01, M02, M04'.split(', ')
val_people = 'F10, M07'.split(', ')
test_people = 'F11, M08'.split(', ')
print(train_people)
print(val_people)
print(test_people)

classes_num = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
word_ids = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
classes_dict = dict(zip(classes_num, classes))
print(classes_num)
print(word_ids)
print(classes_dict)

['F01', 'F02', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'M01', 'M02', 'M04']
['F10', 'M07']
['F11', 'M08']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
{'01': 'Begin', '02': 'Choose', '03': 'Connection', '04': 'Navigation', '05': 'Next', '06': 'Previous', '07': 'Start', '08': 'Stop', '09': 'Hello', '10': 'Web'}


In [4]:
def show_img(img, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(img)
    plt.imshow(img)

In [5]:
all_images = []

for classi in classes_dict.values():
    for i in sorted(glob.glob('data/train/' + classi + '/*.jpg')):
        all_images.append(i)
    print(len(all_images))

4176
9648
13680
17064
21528
26640
32544
37656
41616
46872


In [6]:
video_names = []
frame_nums = []

for img in all_images:
    img_name = img.split('/')[-1].split('.')[0]
    video_name = img_name.split('_color')[0]
    video_names.append(video_name)
    frame_num = img_name[-3:]
    frame_nums.append(frame_num)

print(len(video_names))
print(len(frame_nums))

46872
46872


In [7]:
video_names_uniq = list(sorted(set(video_names)))

frame_nums_uniq = []

for i in range(len(video_names)):
    if i < len(video_names)-1:
        if video_names[i] == video_names[i+1]:
            pass
        else:
            frame_nums_uniq.append(frame_nums[i])
    else:
        frame_nums_uniq.append(frame_nums[-1])

print(len(video_names_uniq))
print(len(frame_nums_uniq))
print(video_names_uniq[:3])
print(frame_nums_uniq[:3])
print(video_names_uniq[-3:])
print(frame_nums_uniq[-3:])

3906
3906
['01_F01_01', '01_F01_03', '01_F01_04']
['012', '012', '012']
['[vert_flip]10_M04_08', '[vert_flip]10_M04_09', '[vert_flip]10_M04_10']
['012', '012', '012']


In [8]:
vids_and_frames = {}

for i in range(len(video_names_uniq)):
        vids_and_frames[video_names_uniq[i]] = frame_nums_uniq[i]

print(len(vids_and_frames))
print(vids_and_frames)

3906
{'01_F01_01': '012', '01_F01_03': '012', '01_F01_04': '012', '01_F01_05': '012', '01_F01_06': '012', '01_F01_07': '012', '01_F01_08': '012', '01_F01_09': '012', '01_F01_10': '012', '01_F05_02': '012', '01_F05_04': '012', '01_F05_08': '012', '01_F05_09': '012', '01_F06_01': '012', '01_F07_01': '012', '01_F07_02': '012', '01_F07_03': '012', '01_F07_04': '012', '01_F07_05': '012', '01_F07_06': '012', '01_F07_07': '012', '01_F07_08': '012', '01_F07_10': '012', '01_F08_01': '012', '01_F08_03': '012', '01_F08_04': '012', '01_F08_05': '012', '01_F08_06': '012', '01_F08_09': '012', '01_F08_10': '012', '01_F09_01': '012', '01_F09_02': '012', '01_F09_03': '012', '01_F09_04': '012', '01_F09_05': '012', '01_F09_06': '012', '01_F09_07': '012', '01_F09_08': '012', '01_F09_10': '012', '01_M01_01': '012', '01_M01_02': '012', '01_M01_03': '012', '01_M01_04': '012', '01_M01_05': '012', '01_M01_06': '012', '01_M01_07': '012', '01_M01_08': '012', '01_M01_09': '012', '01_M01_10': '012', '01_M04_01': '

In [9]:
frames = glob.glob('data/train/Begin/*.jpg')

video_names = []
frame_nums = []

for frame in frames:
    img_name = frame.split('/')[-1].split('.')[0]
    video_name = img_name.split('_color')[0]
    video_names.append(video_name)
    frame_num = img_name[-3:]
    frame_nums.append(frame_num)

print(len(video_names))
print(len(frame_nums))

4176
4176


In [10]:
for i in range(len(video_names)):
    video_names[i] = str(i) + '_' + video_names[i]

In [ ]:
Begin, Choose, Connection, Navigation, Next, Previous, Start, Stop, Hello, Web

In [29]:
frames = sorted(glob.glob('data/train/Begin/*.jpg'))
frames2 = sorted(glob.glob('data/train/Choose/*.jpg'))
frames3 = sorted(glob.glob('data/train/Connection/*.jpg'))
frames4 = sorted(glob.glob('data/train/Navigation/*.jpg'))
frames5 = sorted(glob.glob('data/train/Next/*.jpg'))
frames6 = sorted(glob.glob('data/train/Previous/*.jpg'))
frames7 = sorted(glob.glob('data/train/Start/*.jpg'))
frames8 = sorted(glob.glob('data/train/Stop/*.jpg'))
frames9 = sorted(glob.glob('data/train/Hello/*.jpg'))
frames10 = sorted(glob.glob('data/train/Web/*.jpg'))

In [30]:
print(len(frames))
print(len(frames2))
print(len(frames3))
print(len(frames4))
print(len(frames5))
print(len(frames6))
print(len(frames7))
print(len(frames8))
print(len(frames9))
print(len(frames10))

4176
5472
4032
3384
4464
5112
5904
5112
3960
5256


In [84]:
#randomize train data

for classi in classes:
    matched = 0
    video_names = []
    frames = glob.glob('data/train/' + classi + '/' + '*.jpg')
    
    for frame in frames:
        img_name = frame.split('/')[-1].split('.')[0]
        video_name = img_name.split('_color')[0]
        video_names.append(video_name)
    
    video_names_uniq = list(set(video_names))
    
    for i in range(len(video_names_uniq)):
        video_names_uniq[i] = str(i).zfill(6) + '_' + video_names_uniq[i]
    
    frames = sorted(glob.glob('data/train/' + classi + '/' + '*.jpg'))
    
    for i in range(len(video_names_uniq)):
        for frame in frames:
            if video_names_uniq[i].split(str(i).zfill(6) + '_')[1] == frame.split('/')[-1].split('.')[0].split('_color')[0]:
                os.rename(frame, frame.split('/')[0] + '/' + frame.split('/')[1] + '/' + frame.split('/')[2] + '/' + str(i).zfill(6) + '_' + frame.split('/')[-1])

In [95]:
train_image = []
train_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/train/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        if ('noised' in images[i] or 'rand_contr' in images[i]
        or 'vert_flip' in images[i] or 'hor_flip' in images[i]):
            #print(images[i].split('/')[3])
            #print(classes_dict[images[i].split('/')[3].split(']')[-1][:2]])
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split(']')[-1][:2]])
        else:
            #print(images[i].split('/')[3])
            #print(images[i].split('/')[3].split('_')[1])
            #print(classes_dict[images[i].split('/')[3].split('_')[1]])
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split('_')[1]])
        
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

train_data.to_csv('data/miracl/train_new.csv',header=True, index=False)

100%|██████████| 10/10 [00:00<00:00, 30.05it/s]


In [96]:
train = pd.read_csv('data/miracl/train_new.csv')
train.head()

,image,class
0,000000_[vert_flip]01_M04_02_color_001.jpg,Begin
1,000000_[vert_flip]01_M04_02_color_002.jpg,Begin
2,000000_[vert_flip]01_M04_02_color_003.jpg,Begin
3,000000_[vert_flip]01_M04_02_color_004.jpg,Begin
4,000000_[vert_flip]01_M04_02_color_005.jpg,Begin


In [97]:
train.tail()

,image,class
46867,000437_10_F05_08_color_008.jpg,Web
46868,000437_10_F05_08_color_009.jpg,Web
46869,000437_10_F05_08_color_010.jpg,Web
46870,000437_10_F05_08_color_011.jpg,Web
46871,000437_10_F05_08_color_012.jpg,Web


In [98]:
len(train)

46872

In [99]:
y_tr = train['class']
y_tr_dummy = pd.get_dummies(y_tr)

In [100]:
val_image = []
val_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/validation/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        val_image.append(images[i].split('/')[3])
        val_class.append(classes_dict[images[i].split('/')[3].split('_')[0]])
val_data = pd.DataFrame()
val_data['image'] = val_image
val_data['class'] = val_class

val_data.to_csv('data/miracl/val_new.csv',header=True, index=False)

100%|██████████| 10/10 [00:00<00:00, 652.60it/s]


In [101]:
val = pd.read_csv('data/miracl/val_new.csv')
val.head()

,image,class
0,01_F10_01_color_001.jpg,Begin
1,01_F10_01_color_002.jpg,Begin
2,01_F10_01_color_003.jpg,Begin
3,01_F10_01_color_004.jpg,Begin
4,01_F10_01_color_005.jpg,Begin


In [102]:
val.tail()

,image,class
1505,10_M07_10_color_006.jpg,Web
1506,10_M07_10_color_007.jpg,Web
1507,10_M07_10_color_008.jpg,Web
1508,10_M07_10_color_009.jpg,Web
1509,10_M07_10_color_010.jpg,Web


In [103]:
len(val)

1510

In [104]:
y_val = val['class']
y_val_dummy = pd.get_dummies(y_val)

In [0]:
from keras.models import Model

In [0]:
video = Input(shape=(22, 50, 100, 3))

cnn_base = MobileNetV2(input_shape=(50, 100, 3), include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)

cnn = Model(input=cnn_base.input, output=cnn_out)
encoded_frames = TimeDistributed(cnn)(video)

encoded_sequence = LSTM(54)(encoded_frames)
outputs = Dense(10, activation="softmax")(encoded_sequence)

model = Model([video], outputs)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`
  


In [0]:
adam2 = optimizers.Adam(lr=0.0001)
save_weights = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 22, 50, 100, 3)    0         
_________________________________________________________________
time_distributed_46 (TimeDis (None, 22, 1280)          2257984   
_________________________________________________________________
lstm_25 (LSTM)               (None, 54)                288360    
_________________________________________________________________
dense_47 (Dense)             (None, 10)                550       
Total params: 2,546,894
Trainable params: 2,512,782
Non-trainable params: 34,112
_________________________________________________________________


In [0]:
history = model.fit_generator(train_data, epochs=20, steps_per_epoch=194, 
                              validation_data=val_data, validation_steps=35, shuffle=False)

NameError: ignored

In [105]:
from tweaked_ImageGenerator_v2 import ImageDataGenerator

In [106]:
datagen = ImageDataGenerator()
train_data = datagen.flow_from_directory('data/train', target_size=(50, 100), batch_size=2, frames_per_step=22, shuffle=False)

Found 46872 images belonging to 10 classes.


In [107]:
val_data = datagen.flow_from_directory('data/validation', target_size=(50, 100), batch_size=2, frames_per_step=22, shuffle=False)

Found 1510 images belonging to 10 classes.


In [108]:
import math

print(math.ceil(8520/(2*22)))
print(math.ceil((1510/(2*22))))

194
35


In [118]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

2.1.0
2.2.4-tf


In [109]:
model_small = Sequential()

model_small.add(ZeroPadding3D(padding=(1, 2, 2), input_shape=(22, 50, 100, 3)))
model_small.add(Conv3D(32, (3, 5, 5), strides=(1, 2, 2), activation='relu'))
model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(Dropout(.3))

model_small.add(ZeroPadding3D(padding=(1, 2, 2)))
model_small.add(Conv3D(64, (3, 5, 5), strides=(1, 1, 1), activation='relu'))
model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(Dropout(.3))

model_small.add(ZeroPadding3D(padding=(1, 1, 1)))
model_small.add(Conv3D(96, (3, 3, 3), strides=(1, 2, 2), activation='relu'))
model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(Dropout(.3))

model_small.add(TimeDistributed(Flatten()))

#model_small.add(Bidirectional(GRU(256, activation='relu', return_sequences=True)))
#model_small.add(Bidirectional(GRU(256, activation='relu', return_sequences=False)))
model_small.add(Bidirectional(LSTM(256, activation='relu', return_sequences=False)))

model_small.add(Dense(10, activation='softmax'))

In [110]:
from keras.callbacks import ModelCheckpoint
adam2 = optimizers.Adam(lr=0.0001)
save_weights = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model_small.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])
model_small.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding3d_1 (ZeroPaddin (None, 24, 54, 104, 3)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 22, 25, 50, 32)    7232      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 22, 12, 25, 32)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 12, 25, 32)    0         
_________________________________________________________________
zero_padding3d_2 (ZeroPaddin (None, 24, 16, 29, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 22, 12, 25, 64)    153664    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 22, 6, 12, 64)    

In [111]:
history = model_small.fit_generator(train_data, epochs=20, steps_per_epoch=130, 
                              validation_data=val_data, validation_steps=23, shuffle=False)

Epoch 1/20
  4/130 [..............................] - ETA: 31:38 - loss: 5.3201 - accuracy: 0.9545

KeyboardInterrupt: 

In [0]:
history = model_small.fit_generator(train_data, epochs=20, steps_per_epoch=130, 
                              validation_data=val_data, validation_steps=23, 
                              callbacks=[save_weights], shuffle=True)